https://adventofcode.com/2022/day/18

In [41]:
from collections import defaultdict

from itertools import chain
import networkx as nx

In [1]:
with open("data/18.txt") as fh:
    data = fh.read()

In [2]:
testdata = """\
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
"""

In [5]:
def load_data(data):
    S = set()
    for line in data.strip().split():
        p = tuple(map(int, line.split(",")))
        S.add(p)
    return S

In [6]:
load_data(testdata)

{(1, 2, 2),
 (1, 2, 5),
 (2, 1, 2),
 (2, 1, 5),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 6),
 (2, 3, 2),
 (2, 3, 5),
 (3, 2, 2),
 (3, 2, 5)}

In [7]:
def find_neighbors(p, grid):
    x, y, z = p
    for nabe in [
        (x+1, y, z),
        (x-1, y, z),
        (x, y+1, z),
        (x, y-1, z),
        (x, y, z+1),
        (x, y, z-1)
    ]:
        if nabe in grid:
            yield nabe


In [45]:
def free_faces(p, grid):
    x, y, z = p
    for nabe in [
        (x+1, y, z),
        (x-1, y, z),
        (x, y+1, z),
        (x, y-1, z),
        (x, y, z+1),
        (x, y, z-1)
    ]:
        if nabe not in grid:
            yield nabe


In [23]:
def surface_area(grid):
    return sum(sum(1 for _ in free_faces(p, grid)) for p in grid)

In [24]:
grid = load_data(testdata)
surface_area(grid)

64

Part 1

In [26]:
grid = load_data(data)
surface_area(grid)

3412

Part 2
Find open faces connected to outside.

In [70]:
%%time
lava = load_data(data)
xmax = max(x for (x, y, z) in lava)
xmin = min(x for (x, y, z) in lava)
ymax = max(y for (x, y, z) in lava)
ymin = min(y for (x, y, z) in lava)
zmax = max(z for (x, y, z) in lava)
zmin = min(z for (x, y, z) in lava)

spaces = set()
for p in ((x,y,z) for x in range(xmin-1, xmax+2) for y in range(ymin-1, ymax+2) for z in range(zmin-1, zmax+2)):
    if p not in lava:
        spaces.add(p)
G = nx.Graph()
for sp in spaces:
    for nabe in find_neighbors(sp, spaces):
        G.add_edge(sp, nabe)
    x, y, z = sp
    if not all([xmin-1 < x < xmax+1, ymin-1 < y < ymax+1, zmin-1 < z < zmax+1]):
        G.add_edge(sp, "outside")

outside_component = nx.node_connected_component(G, "outside")
surface_area = 0
for p in lava:
    for sp in free_faces(p, lava):
        if sp in outside_component:
            surface_area += 1
surface_area

CPU times: user 106 ms, sys: 7.81 ms, total: 114 ms
Wall time: 113 ms


2018